In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
from os import listdir
from os.path import isfile, join

In [3]:
drive.mount('/gdrive')
%cd /gdrive
%cd MyDrive
%cd mgr
files = [f for f in listdir(f"capital-property-structure/") if '.xlsx' in f]
#files = ["RELPOL.xlsx"]
get_filename = lambda file: file.split('/')[-1][:-5]
company_names = list(map(get_filename, files))

Mounted at /gdrive
/gdrive
/gdrive/MyDrive
/gdrive/MyDrive/mgr


In [4]:
df_sector = pd.DataFrame(columns=['Nazwa firmy','Sektor'])
for file in files:
  company_name = get_filename(file)
  df = pd.read_excel(f"capital-property-structure/{file}", sheet_name='Info')
  df = df = df.iloc[:, -2:]
  df = df.T
  df.columns = df.iloc[0]
  df = df.drop(df.index[0])
  new_row = pd.DataFrame({'Sektor': df.loc[:,"Sektor"].values,
                        'Nazwa firmy': [company_name]})
  df_sector = pd.concat([df_sector,new_row],ignore_index=True)

df_sector.loc[df_sector['Nazwa firmy'] == 'BMW', 'Sektor'] = "motoryzacja"
df_sector.loc[df_sector['Nazwa firmy'] == 'MERCEDES', 'Sektor'] = "motoryzacja"
df_sector.loc[df_sector['Nazwa firmy'] == 'UFGAMES', 'Sektor'] = "gry"
df_sector.loc[df_sector['Nazwa firmy'] == '3LPSA', 'Sektor'] = "transport i logistyka - pozostałe"


In [5]:
dataframes =[]
for file in files:
  company_name = get_filename(file)
  df = pd.read_excel(f"capital-property-structure/{file}", sheet_name='YC')
  df = df.T
  df.columns = df.iloc[1]
  df = df.drop(df.index[0:14])
  df = df.reset_index()
  df.columns = ["Okres obrachunkowy"] + df.columns[1:].tolist()
  df.insert(0, "Nazwa firmy", np.full(df.shape[0], company_name), True)
  df = df[df.columns[0:1].tolist() + df.columns[29:93].tolist()]
  df = df.loc[:,~df.columns.duplicated(keep='first')].copy()
  df = df.fillna(0)
  df = df[(df["AKTYWA"] != 0) & (df["PASYWA"] != 0)]

  df = pd.merge(df,df_sector, on='Nazwa firmy')
  dataframes.append(df)
  df = pd.concat(dataframes, ignore_index=True)

all_columns = df.columns
df = df.loc[:, (df != 0).any(axis=0)]
after_drop_columns = df.columns
result = [x for x in all_columns if x not in after_drop_columns]
df_with_sector = df.copy()


In [6]:
import re
%cd ./market-value
%ls
files_market_value = [f for f in listdir(".") if '.xls' in f]
get_year = lambda file: re.findall(r'\d+', file.split('/')[-1][:-5])[0]
years = list(map(get_year, files_market_value))

tab_name= {
    '2014': 'Tab 17',
    '2015': 'Tab 17',
    '2016': 'Tab 17',
    '2017': 'Tab 17',
    '2018': 'Tab 12',
    '2019': 'Tab 12',
    '2020': 'Tab 12',
    '2021': 'Tab 12',
    '2022': 'Tab 12'
}

dataframes_market_value =[]
for file in files_market_value:
  year = get_year(file)
  df_market_value_manual = pd.read_excel(f"{file}", sheet_name=tab_name[year])
  df_market_value_manual.columns = df_market_value_manual.iloc[1]
  df_market_value_manual = df_market_value_manual.iloc[3:]
  df_market_value_manual = df_market_value_manual.loc[:, df_market_value_manual.columns.str.contains('Company|Market value', case=False, na=False)]
  df_market_value_manual.columns = ['Nazwa firmy', 'Wartość rynkowa']
  df_market_value_manual['Rok'] = int(year)-1

  dataframes_market_value.append(df_market_value_manual)

df_market_value_manual = pd.concat(dataframes_market_value, ignore_index=True)

%cd ../market-value-exported
%ls

df_market_value_exported = pd.read_csv("./market_value.csv")
df_market_value_exported = df_market_value_exported.rename(columns={'Name': 'Nazwa firmy', 'Market value': 'Wartość rynkowa', 'Period end': 'Rok'})
df_market_value_exported['Rok'] = df_market_value_exported['Rok'].apply(lambda x: x[:4])
df_market_value_exported['Wartość rynkowa'] = df_market_value_exported['Wartość rynkowa'] / 1000000


df_market_value_manual['Rok'] = df_market_value_manual['Rok'].astype(int)
df_market_value_exported['Rok'] = df_market_value_exported['Rok'].astype(int)
df_market_value = pd.concat([df_market_value_manual, df_market_value_exported]).drop_duplicates(subset=['Rok', 'Nazwa firmy'])

df_with_market_value = df_with_sector.copy()
df_with_market_value['Data bilansowa'] = pd.to_datetime(df_with_market_value['Data bilansowa']).dt.year
df_with_market_value = df_with_market_value.rename(columns={'Data bilansowa': 'Rok'})
df_with_market_value = df_with_market_value.merge(df_market_value, on=['Nazwa firmy', 'Rok'], how='inner')

# Połączenie ramek danych, zaznaczenie, które wiersze pochodzą z df1
df1 = df_with_sector.copy()
df1['Data bilansowa'] = pd.to_datetime(df1['Data bilansowa']).dt.year
df1 = df1.rename(columns={'Data bilansowa': 'Rok'})
df2 = df_with_market_value.copy()
merged_df = df1.merge(df2, on=['Nazwa firmy', 'Rok'], how='left', indicator=True)

missing_rows_df1 = merged_df[merged_df['_merge'] == 'left_only'].drop(columns='_merge')
missing_rows_df1.head()

/gdrive/MyDrive/mgr/market-value
'Rocznik 1992.pdf'  'Rocznik 2000.pdf'  'Rocznik 2008.pdf'      Rocznik_2016__GR.xls
'Rocznik 1993.pdf'  'Rocznik 2001.pdf'  'Rocznik 2009.pdf'      Rocznik_2017_GR.xls
'Rocznik 1994.pdf'  'Rocznik 2002.pdf'  'Rocznik 2010.pdf'      Rocznik_2018_GR.xls
'Rocznik 1995.pdf'  'Rocznik 2003.pdf'  'Rocznik 2011.pdf'      Rocznik_2019_GR.xls
'Rocznik 1996.pdf'  'Rocznik 2004.pdf'  'Rocznik 2012.pdf'     'Rocznik_2020_GR (5).xls'
'Rocznik 1997.pdf'  'Rocznik 2005.pdf'  'Rocznik 2013.pdf'      Rocznik_2021-GR.xls
'Rocznik 1998.pdf'  'Rocznik 2006.pdf'   Rocznik_2014__GR.xls   Rocznik_2022_GR.xls
'Rocznik 1999.pdf'  'Rocznik 2007.pdf'   Rocznik_2015__GR.xls
WARNING *** file size (1183124) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
/gdrive/MyDrive/mgr/market-value-exported
market_value.csv
Wiersze brakujące w df2:
171125


,Nazwa firmy,Rok,AKTYWA_x,Aktywa trwałe_x,Rzeczowe aktywa trwałe_x,Aktywa z tytułu poszukiwania i oceny zasobów mineralnych_x,Wartości niematerialne i prawne_x,Wartość firmy_x,Nieruchomości inwestycyjne_x,Aktywa z tytułu prawa do użytkowania_x,...,Zobowiązania z tytułu obligacji_y,Zobowiązania z tytułu leasingu_y,Zobowiązania handlowe_y,Świadczenia pracownicze_y,Zobowiązania z tytułu bieżącego podatku_y,Rezerwy_y,Pozostałe zobowiązania_y,Rozliczenia międzyokresowe (zobowiązania)_y,Sektor_y,Wartość rynkowa
0,AGROWILL,2008,347018.0,285091.0,161103.0,0.0,5246.0,0.0,82364.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AGROWILL,2009,272238.0,242637.0,146960.0,0.0,2961.0,0.0,82364.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AGROWILL,2010,288964.0,242891.0,140442.0,0.0,2916.0,0.0,81794.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AGROWILL,2017,148548.0,99131.0,85235.0,0.0,839.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,AGROWILL,2018,171890.0,111938.0,92892.0,0.0,2427.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_diff = df_with_market_value.copy()
df_diff.sort_values(by=['Nazwa firmy', 'Rok'])

df_diff['Rok poprzedni'] = df_diff.groupby('Nazwa firmy')['Rok'].shift(1)
columns_to_diff = [col for col in df_diff.columns if col not in ['Nazwa firmy', 'Rok','Rok poprzedni', 'Sektor']]
new_columns = []

for column in columns_to_diff:
    new_columns.append(column)
    new_market_value = df_diff[column]
    old_market_value = df_diff.groupby('Nazwa firmy')[column].shift(1)
    df_diff[column] = old_market_value - new_market_value

df_diff = df_diff.dropna()
df_diff = df_diff[['Nazwa firmy', 'Sektor'] + new_columns + ['Rok']]
df_diff = df_diff.reset_index(drop=True)

columns_not_to_scale = ['Nazwa firmy', 'Sektor', 'Rok']
columns_to_scale = df_diff.columns.difference(columns_not_to_scale)
df_diff_filtered = df_diff[columns_to_scale]

136630
121940


In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_scaled = df_diff.copy()
columns_not_to_scale = ['Nazwa firmy', 'Sektor', 'Rok']
columns_to_scale = df_scaled.columns.difference(columns_not_to_scale)
df_scaled[columns_to_scale] = scaler.fit_transform(df_scaled[columns_to_scale])

120064


,Nazwa firmy,Sektor,AKTYWA,Aktywa trwałe,Rzeczowe aktywa trwałe,Aktywa z tytułu poszukiwania i oceny zasobów mineralnych,Wartości niematerialne i prawne,Wartość firmy,Nieruchomości inwestycyjne,Aktywa z tytułu prawa do użytkowania,...,Zobowiązania z tytułu obligacji,Zobowiązania z tytułu leasingu,Zobowiązania handlowe,Świadczenia pracownicze,Zobowiązania z tytułu bieżącego podatku,Rezerwy,Pozostałe zobowiązania,Rozliczenia międzyokresowe (zobowiązania),Wartość rynkowa,Rok
0,AGROWILL,produkcja rolna i rybołówstwo,0.630051,0.728761,0.761993,0.724443,0.539859,0.42624,0.892353,0.881533,...,0.530705,0.864402,0.357319,0.726868,0.703849,0.571228,0.915286,0.542206,0.412486,2012
1,AGROWILL,produkcja rolna i rybołówstwo,0.630139,0.728855,0.762023,0.724443,0.539865,0.42624,0.925601,0.881533,...,0.530705,0.864402,0.357324,0.726868,0.703842,0.571228,0.915384,0.542206,0.412061,2013
2,AGROWILL,produkcja rolna i rybołówstwo,0.630066,0.728779,0.762019,0.724443,0.539859,0.42624,0.888219,0.881533,...,0.530705,0.862386,0.357325,0.726868,0.703843,0.571228,0.915300,0.542206,0.411001,2014
3,AGROWILL,produkcja rolna i rybołówstwo,0.630053,0.728759,0.761992,0.724443,0.539860,0.42624,0.892170,0.881533,...,0.530705,0.863887,0.357325,0.726868,0.703845,0.571228,0.915342,0.542206,0.412785,2015
4,AGROWILL,produkcja rolna i rybołówstwo,0.630066,0.728780,0.762007,0.724443,0.539860,0.42624,0.897123,0.881533,...,0.530705,0.863514,0.357322,0.726868,0.703851,0.571228,0.915320,0.542206,0.410062,2016


In [13]:
df_scaled.head()

,Nazwa firmy,Sektor,AKTYWA,Aktywa trwałe,Rzeczowe aktywa trwałe,Aktywa z tytułu poszukiwania i oceny zasobów mineralnych,Wartości niematerialne i prawne,Wartość firmy,Nieruchomości inwestycyjne,Aktywa z tytułu prawa do użytkowania,...,Zobowiązania z tytułu obligacji,Zobowiązania z tytułu leasingu,Zobowiązania handlowe,Świadczenia pracownicze,Zobowiązania z tytułu bieżącego podatku,Rezerwy,Pozostałe zobowiązania,Rozliczenia międzyokresowe (zobowiązania),Wartość rynkowa,Rok
0,AGROWILL,produkcja rolna i rybołówstwo,0.630051,0.728761,0.761993,0.724443,0.539859,0.42624,0.892353,0.881533,...,0.530705,0.864402,0.357319,0.726868,0.703849,0.571228,0.915286,0.542206,0.412486,2012
1,AGROWILL,produkcja rolna i rybołówstwo,0.630139,0.728855,0.762023,0.724443,0.539865,0.42624,0.925601,0.881533,...,0.530705,0.864402,0.357324,0.726868,0.703842,0.571228,0.915384,0.542206,0.412061,2013
2,AGROWILL,produkcja rolna i rybołówstwo,0.630066,0.728779,0.762019,0.724443,0.539859,0.42624,0.888219,0.881533,...,0.530705,0.862386,0.357325,0.726868,0.703843,0.571228,0.915300,0.542206,0.411001,2014
3,AGROWILL,produkcja rolna i rybołówstwo,0.630053,0.728759,0.761992,0.724443,0.539860,0.42624,0.892170,0.881533,...,0.530705,0.863887,0.357325,0.726868,0.703845,0.571228,0.915342,0.542206,0.412785,2015
4,AGROWILL,produkcja rolna i rybołówstwo,0.630066,0.728780,0.762007,0.724443,0.539860,0.42624,0.897123,0.881533,...,0.530705,0.863514,0.357322,0.726868,0.703851,0.571228,0.915320,0.542206,0.410062,2016


In [15]:
columns_to_exclude = ['Nazwa firmy','Wartość rynkowa', 'Sektor', 'Rok']
columns_to_include = [col for col in df_scaled.columns if col not in columns_to_exclude]
df_scaled_numerical = df_scaled[columns_to_include]

df_scaled_numerical.head()

,AKTYWA,Aktywa trwałe,Rzeczowe aktywa trwałe,Aktywa z tytułu poszukiwania i oceny zasobów mineralnych,Wartości niematerialne i prawne,Wartość firmy,Nieruchomości inwestycyjne,Aktywa z tytułu prawa do użytkowania,Inwestycje w jednostki powiązane,Długoterminowe aktywa finansowe,...,Zobowiązania z tytułu instrumentów pochodnych,Zobowiązania finansowe (kredyty i pożyczki),Zobowiązania z tytułu obligacji,Zobowiązania z tytułu leasingu,Zobowiązania handlowe,Świadczenia pracownicze,Zobowiązania z tytułu bieżącego podatku,Rezerwy,Pozostałe zobowiązania,Rozliczenia międzyokresowe (zobowiązania)
0,0.630051,0.728761,0.761993,0.724443,0.539859,0.42624,0.892353,0.881533,0.187481,0.798034,...,0.917021,0.691338,0.530705,0.864402,0.357319,0.726868,0.703849,0.571228,0.915286,0.542206
1,0.630139,0.728855,0.762023,0.724443,0.539865,0.42624,0.925601,0.881533,0.187481,0.798024,...,0.917021,0.691399,0.530705,0.864402,0.357324,0.726868,0.703842,0.571228,0.915384,0.542206
2,0.630066,0.728779,0.762019,0.724443,0.539859,0.42624,0.888219,0.881533,0.187481,0.798022,...,0.917021,0.691319,0.530705,0.862386,0.357325,0.726868,0.703843,0.571228,0.915300,0.542206
3,0.630053,0.728759,0.761992,0.724443,0.539860,0.42624,0.892170,0.881533,0.187481,0.798022,...,0.917021,0.691319,0.530705,0.863887,0.357325,0.726868,0.703845,0.571228,0.915342,0.542206
4,0.630066,0.728780,0.762007,0.724443,0.539860,0.42624,0.897123,0.881533,0.187480,0.798028,...,0.917021,0.691346,0.530705,0.863514,0.357322,0.726868,0.703851,0.571228,0.915320,0.542206


In [16]:
import numpy as np
from sklearn.metrics import auc


def em(t, t_max, volume_support, s_unif, s_X, n_generated):
    EM_t = np.zeros(t.shape[0])
    n_samples = s_X.shape[0]
    s_X_unique = np.unique(s_X)
    EM_t[0] = 1.
    for u in s_X_unique:
        # if (s_unif >= u).sum() > n_generated / 1000:
        EM_t = np.maximum(EM_t, 1. / n_samples * (s_X > u).sum() -
                          t * (s_unif > u).sum() / n_generated
                          * volume_support)
    amax = np.argmax(EM_t <= t_max) + 1
    if amax == 1:
        print('\n failed to achieve t_max \n')
        amax = -1
    AUC = auc(t[:amax], EM_t[:amax])
    return AUC, EM_t, amax


def mv(axis_alpha, volume_support, s_unif, s_X, n_generated):
    n_samples = s_X.shape[0]
    s_X_argsort = s_X.argsort()
    mass = 0
    cpt = 0
    u = s_X[s_X_argsort[-1]]
    mv = np.zeros(axis_alpha.shape[0])
    for i in range(axis_alpha.shape[0]):
        # pdb.set_trace()
        while mass < axis_alpha[i]:
            cpt += 1
            u = s_X[s_X_argsort[-cpt]]
            mass = 1. / n_samples * cpt  # sum(s_X > u)
        mv[i] = float((s_unif >= u).sum()) / n_generated * volume_support
    return auc(axis_alpha, mv), mv

In [17]:
import numpy as np

# import matplotlib.pyplot as plt
# for the cluster to save the fig:
import matplotlib

matplotlib.use('Agg')

import sys

from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.utils import shuffle as sh


# framework: outlier or novelty detection
novelty_detection = True

# parameters of the algorithm:
averaging = 50
max_features = 5
n_generated = 100000
alpha_min = 0.9
alpha_max = 0.999
t_max = 0.95
ocsvm_max_train = 10000

np.random.seed(1)

datasets = [
  # {'key': 'HANDEL_aktywa', 'value': HANDEL_aktywa.to_numpy()},
  # {'key': 'HANDEL_pasywa', 'value': HANDEL_aktywa.to_numpy()},
  # {'key': 'HANDEL_pasywa_aktywa', 'value': HANDEL_pasywa_aktywa.to_numpy()},
  # {'key': 'BUILDING_aktywa', 'value': BUILDING_aktywa.to_numpy()},
  # {'key': 'BUILDING_pasywa', 'value': BUILDING_aktywa.to_numpy()},
  # {'key': 'BUILDING_pasywa_aktywa', 'value': BUILDING_pasywa_aktywa.to_numpy()},
  # {'key': 'SHOPS_aktywa', 'value': SHOPS_aktywa.to_numpy()},
  # {'key': 'SHOPS_pasywa', 'value': SHOPS_aktywa.to_numpy()},
  # {'key': 'SHOPS_pasywa_aktywa', 'value': SHOPS_pasywa_aktywa.to_numpy()},
  {'key': 'all_aktywa', 'value': df_scaled_numerical.to_numpy()},
  # {'key': 'RELPOL_aktywa', 'value': RELPOL_aktywa.to_numpy()},
  # {'key': 'RELPOL_pasywa', 'value': RELPOL_pasywa.to_numpy()},
  # {'key': 'RELPOL_pasywa_aktywa', 'value': RELPOL_pasywa_aktywa.to_numpy()},
]

for dat in datasets:
    # loading and vectorization
    name = dat['key']
    X = dat['value']

    n_samples, n_features = np.shape(X)
    n_samples_train = n_samples // 2
    n_samples_test = n_samples - n_samples_train

    X_train = X[:n_samples_train, :]
    X_test = X[n_samples_train:, :]

    n_samples, n_features = np.shape(X)
    n_samples_train = n_samples // 2
    n_samples_test = n_samples - n_samples_train

    X_train = X[:n_samples_train, :]


    iforest = IsolationForest(
        n_estimators=50,
        max_samples='auto',
        max_features=1.0,
        random_state=1234
    )

    iforest1 = IsolationForest(
        n_estimators=50,
        max_samples='auto',
        max_features=0.95,
        random_state=1234
    )


    iforest2 = IsolationForest(
        n_estimators=50,
        max_samples='auto',
        max_features=0.8,
        random_state=1234
    )


    iforest3 = IsolationForest(
        n_estimators=50,
        max_samples='auto',
        max_features=0.6,
        random_state=1234
    )


    iforest4 = IsolationForest(
        n_estimators=50,
        max_samples='auto',
        max_features=0.3,
        random_state=1234
    )

    # lof = LocalOutlierFactor(novelty=True)
    # ocsvm = OneClassSVM()

    n_samples, n_features = X_test.shape
    em_iforest, mv_iforest = 0, 0
    em_iforest1, mv_iforest1 = 0, 0
    em_iforest2, mv_iforest2 = 0, 0
    em_iforest3, mv_iforest3 = 0, 0
    em_iforest4, mv_iforest4 = 0, 0
    # em_lof, mv_lof = 0, 0
    # em_ocsvm, mv_ocsvm = 0, 0
    nb_exp = 0
    while nb_exp < averaging:
        features = sh(np.arange(n_features))[:max_features]
        X_train_ = X_train[:, features]
        X_ = X_test[:, features]

        lim_inf = X_.min(axis=0)
        lim_sup = X_.max(axis=0)
        volume_support = (lim_sup - lim_inf).prod()
        if volume_support > 0:
            nb_exp += 1
            t = np.arange(0, 100 / volume_support, 0.001 / volume_support)
            axis_alpha = np.arange(alpha_min, alpha_max, 0.001)
            unif = np.random.uniform(lim_inf, lim_sup,
                                     size=(n_generated, max_features))

            iforest.fit(X_train_)
            iforest1.fit(X_train_)
            iforest2.fit(X_train_)
            iforest3.fit(X_train_)
            iforest4.fit(X_train_)

            # lof.fit(X_train_)
            # ocsvm.fit(X_train_[:min(ocsvm_max_train, n_samples_train - 1)])
            print('end of ocsvm training!')
            s_X_iforest = iforest.decision_function(X_)
            s_X_iforest1 = iforest1.decision_function(X_)
            s_X_iforest2 = iforest2.decision_function(X_)
            s_X_iforest3 = iforest3.decision_function(X_)
            s_X_iforest4 = iforest4.decision_function(X_)
            # s_X_lof = lof.decision_function(X_)
            # s_X_ocsvm = ocsvm.decision_function(X_).reshape(1, -1)[0]

            s_unif_iforest = iforest.decision_function(unif)
            s_unif_iforest1 = iforest1.decision_function(unif)
            s_unif_iforest2 = iforest2.decision_function(unif)
            s_unif_iforest3 = iforest3.decision_function(unif)
            s_unif_iforest4 = iforest4.decision_function(unif)
            # s_unif_lof = lof.decision_function(unif)
            # s_unif_ocsvm = ocsvm.decision_function(unif).reshape(1, -1)[0]

            em_iforest += em(t, t_max, volume_support, s_unif_iforest,
                             s_X_iforest, n_generated)[0]
            mv_iforest += mv(axis_alpha, volume_support, s_unif_iforest,
                             s_X_iforest, n_generated)[0]

            em_iforest1 += em(t, t_max, volume_support, s_unif_iforest1,
                             s_X_iforest1, n_generated)[0]
            mv_iforest1 += mv(axis_alpha, volume_support, s_unif_iforest1,
                             s_X_iforest1, n_generated)[0]

            em_iforest2 += em(t, t_max, volume_support, s_unif_iforest2,
                             s_X_iforest2, n_generated)[0]
            mv_iforest2 += mv(axis_alpha, volume_support, s_unif_iforest2,
                             s_X_iforest2, n_generated)[0]

            em_iforest3 += em(t, t_max, volume_support, s_unif_iforest3,
                             s_X_iforest3, n_generated)[0]
            mv_iforest3 += mv(axis_alpha, volume_support, s_unif_iforest3,
                             s_X_iforest3, n_generated)[0]

            em_iforest4 += em(t, t_max, volume_support, s_unif_iforest4,
                             s_X_iforest4, n_generated)[0]
            mv_iforest4 += mv(axis_alpha, volume_support, s_unif_iforest4,
                             s_X_iforest4, n_generated)[0]
            # em_lof += em(t, t_max, volume_support, s_unif_lof, s_X_lof,
            #              n_generated)[0]
            # mv_lof += mv(axis_alpha, volume_support, s_unif_lof,
            #              s_X_lof, n_generated)[0]
            # em_ocsvm += em(t, t_max, volume_support, s_unif_ocsvm,
            #                s_X_ocsvm, n_generated)[0]
            # mv_ocsvm += mv(axis_alpha, volume_support, s_unif_ocsvm,
                          #  s_X_ocsvm, n_generated)[0]

    em_iforest /= averaging
    mv_iforest /= averaging

    em_iforest1 /= averaging
    mv_iforest1 /= averaging

    em_iforest2 /= averaging
    mv_iforest2 /= averaging

    em_iforest3 /= averaging
    mv_iforest3 /= averaging

    em_iforest4 /= averaging
    mv_iforest4 /= averaging

    # em_lof /= averaging
    # mv_lof /= averaging
    # em_ocsvm /= averaging
    # mv_ocsvm /= averaging

    with open(f'wyniki_{name}', 'a') as result:
        result.write('em_iforest = ' + str(em_iforest) + '\n')
        result.write('em_iforest1 = ' + str(em_iforest1) + '\n')
        result.write('em_iforest2 = ' + str(em_iforest2) + '\n')
        result.write('em_iforest3 = ' + str(em_iforest3) + '\n')
        result.write('em_iforest4 = ' + str(em_iforest4) + '\n')
        # result.write('em_lof = ' + str(em_lof) + '\n')
        # result.write('em_ocsvm = ' + str(em_ocsvm) + '\n \n')

        result.write('mv_iforest = ' + str(mv_iforest) + '\n')
        result.write('mv_iforest1 = ' + str(mv_iforest1) + '\n')
        result.write('mv_iforest2 = ' + str(mv_iforest2) + '\n')
        result.write('mv_iforest3 = ' + str(mv_iforest3) + '\n')
        result.write('mv_iforest4 = ' + str(mv_iforest4) + '\n')
        # result.write('mv_lof = ' + str(mv_lof) + '\n')
        # result.write('mv_ocsvm = ' + str(mv_ocsvm) + '\n')


end of ocsvm training!

 failed to achieve t_max 


 failed to achieve t_max 


 failed to achieve t_max 


 failed to achieve t_max 


 failed to achieve t_max 

end of ocsvm training!
end of ocsvm training!

 failed to achieve t_max 


 failed to achieve t_max 


 failed to achieve t_max 


 failed to achieve t_max 

end of ocsvm training!

 failed to achieve t_max 

end of ocsvm training!

 failed to achieve t_max 


 failed to achieve t_max 


 failed to achieve t_max 


 failed to achieve t_max 


 failed to achieve t_max 

end of ocsvm training!

 failed to achieve t_max 

end of ocsvm training!

 failed to achieve t_max 


 failed to achieve t_max 


 failed to achieve t_max 


 failed to achieve t_max 


 failed to achieve t_max 

end of ocsvm training!

 failed to achieve t_max 


 failed to achieve t_max 


 failed to achieve t_max 


 failed to achieve t_max 


 failed to achieve t_max 

end of ocsvm training!

 failed to achieve t_max 


 failed to achieve t_max 


 failed 

In [18]:
print(em_iforest)

40640870442.610405
